In [2]:
from tqdm import tqdm
import pandas as pd
import math
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_ming_title_raw=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=843266982&single=true&output=tsv',
                              sep='\t')

In [4]:
df_ming_title_raw.sample(3)

,LINE #,c_office_id（Dictionary Ser#),Institution 1,Institution 2,Institution 3,c_office_chn,length,c_office_chn_clean,part_time,data_cat,titles to revise,Unnamed: 11,Unnamed: 12,Unnamed: 13,titles to revise.1,Unnamed: 15,title_revised,title_revised_to run
3994,4593,70857;70912,皇族宮廷類 Imperial Family and Royal Court,皇太子與東宮門 Heir Apparent and Eastern Palace,司經局 The Editorial Service,洗馬,2,洗馬,NaN,1,洗馬,NaN,NaN,NaN,洗馬,T,洗馬///,洗馬/
1308,344,1319,中央中樞官署類 The Central Government,六部門 Six Ministries,工部 The Ministry of Works,屯部員外郎,5,屯部員外郎,NaN,1,屯部,員外郎,NaN,NaN,屯部員外郎,T,屯部/員外郎//,屯部/員外郎/
1249,1252,70380,中央輔佐官署類 Central Administration Assistance,秘書門 Secretary Offices,翰林院 The Hanlin Academy,國史院學士,5,國史院學士,NaN,1,國史院,學士,NaN,NaN,國史院學士,T,國史院/學士//,國史院/學士/


Draft.
```Python
for index in tqdm(df_ming_title_raw.index):
    df_ming_title_raw.loc[index, 'concat']='/'.join([str(s).strip() for s in df_ming_title_raw[['titles to revise','Unnamed: 11','Unnamed: 12','Unnamed: 13',]].loc[index]])
df_ming_title_raw['concat']=[s.replace('/nan', '') for s in df_ming_title_raw['concat']]
df_ming_title_refined=df_ming_title_raw[['LINE #','c_office_id（Dictionary Ser#)','Institution 1','Institution 2','Institution 3','data_cat','concat']]
df_ming_title_refined=df_ming_title_refined[df_ming_title_refined.data_cat==1]
```

In [5]:
df_ming_title_refined=df_ming_title_raw[['LINE #','c_office_id（Dictionary Ser#)','Institution 1','Institution 2','Institution 3','data_cat','title_revised_to run']]
df_ming_title_refined['title_revised_to run']=[s.strip('/') for s in df_ming_title_refined['title_revised_to run']]
df_ming_title_refined.rename(columns={'title_revised_to run':'concat'}, inplace=True)
df_ming_title_refined=df_ming_title_refined[(df_ming_title_refined.data_cat==1)|(df_ming_title_refined.data_cat==7)]

In [6]:
df_ming_title_refined.sample(3)

,LINE #,c_office_id（Dictionary Ser#),Institution 1,Institution 2,Institution 3,data_cat,concat
1091,2206,71379,司法監察機構類 Legislation and Censorship,監察門 Censorate,都察院 The Censorate,1,右副都御史
4125,1554,2120,京衛京營與中央軍事官署類 Central and Capital Militaries,京衛門 Military Guards of the Capital City,金吾侍衛親軍都護府 The Imperial Guard and Military Prot...,1,經歷
1588,2363,2565,地方官署類 Regional and Local Governance,京府門 Superior Prefectural Governance,順天府 Shuntian Superior Prefecture,1,稅課司/大使


### Build Classification table (`C_OT_CLS`).

In [7]:
df_C_OT_CLS=pd.DataFrame(columns=['c_ot_cls_chinm','c_ot_cls_engnm','c_ot_cls_desc','c_ot_cls_start','c_ot_cls_end',])

In [8]:
c_ot_cls_list=list(set(df_ming_title_refined['Institution 1'].unique()).union((df_ming_title_refined['Institution 2'].unique())))
c_ot_cls_list=[s.strip() for s in c_ot_cls_list if str(s)!='nan']

In [9]:
c_ot_cls_chinm_list=[str(s).split(' ')[0] for s in c_ot_cls_list]
c_ot_cls_engnm_list=[' '.join(str(s).split(' ')[1:]) for s in c_ot_cls_list]

In [10]:
df_C_OT_CLS['c_ot_cls_chinm']=c_ot_cls_chinm_list
df_C_OT_CLS['c_ot_cls_engnm']=c_ot_cls_engnm_list
df_C_OT_CLS['len']=[len(s) for s in df_C_OT_CLS['c_ot_cls_chinm']]
df_C_OT_CLS.sort_values(by='len', ascending=False, inplace=True)
df_C_OT_CLS.reset_index(inplace=True, drop=True)
df_C_OT_CLS.index.name='c_ot_cls_id'
df_C_OT_CLS.drop('len', axis=1).to_csv('../data_output/C_OT_CLS.tsv', sep='\t', encoding='utf8')

### Build Admin Unit table (`C_OT_ADM`) and Title table (`C_OT_TIT`).

In [11]:
df_C_OT_ADM=pd.DataFrame(columns=['c_ot_adm_chinm','c_ot_adm_engnm','c_ot_adm_desc','c_ot_adm_start','c_ot_adm_end',])
df_C_OT_TIT=pd.DataFrame(columns=['c_ot_tit_chinm','c_ot_tit_engnm','c_ot_tit_desc','c_ot_tit_start','c_ot_tit_end',])

In [12]:
c_ot_adm_list=list(set(df_ming_title_refined['Institution 3'].unique()))
c_ot_adm_list=[s.strip() for s in c_ot_adm_list if str(s)!='nan']

In [13]:
adm_list=[]
for adm_title in [s.split('/') for s in df_ming_title_refined['concat'].unique()]:
    adm_list+=adm_title[:-1]
    
tit_list=[]
for adm_title in [s.split('/') for s in df_ming_title_refined['concat'].unique()]:
    tit_list.append(adm_title[-1])

In [14]:
len(adm_list), len(set(adm_list)), len(tit_list),  len(set(tit_list))

(1967, 942, 2836, 1349)

In [18]:
# Build Admin Unit table.
df_C_OT_ADM['c_ot_adm_chinm']=list(set(adm_list))
df_C_OT_ADM['len']=[len(s) for s in df_C_OT_ADM['c_ot_adm_chinm']]
df_C_OT_ADM.sort_values(by='len', ascending=False, inplace=True)
df_C_OT_ADM.reset_index(inplace=True, drop=True)
df_C_OT_ADM.index.name='c_ot_adm_id'
df_C_OT_ADM.drop('len', axis=1).to_csv('../data_output/C_OT_ADM.tsv', sep='\t', encoding='utf8')
# Build Title table.
df_C_OT_TIT['c_ot_tit_chinm']=list(set(tit_list))
df_C_OT_TIT['len']=[len(s) for s in df_C_OT_TIT['c_ot_tit_chinm']]
df_C_OT_TIT.sort_values(by='len', ascending=False, inplace=True)
df_C_OT_TIT.reset_index(inplace=True, drop=True)
df_C_OT_TIT.index.name='c_ot_tit_id'
df_C_OT_TIT.drop('len', axis=1).to_csv('../data_output/C_OT_TIT.tsv', sep='\t', encoding='utf8')

In [16]:
df_C_OT_ADM.sample(5)

,c_ot_adm_chinm,c_ot_adm_engnm,c_ot_adm_desc,c_ot_adm_start,c_ot_adm_end,len
c_ot_adm_id,,,,,,
895,驃騎,NaN,NaN,NaN,NaN,2
217,蘇州織造,NaN,NaN,NaN,NaN,4
384,兵馬司,NaN,NaN,NaN,NaN,3
186,巡視京營,NaN,NaN,NaN,NaN,4
602,議政,NaN,NaN,NaN,NaN,2


In [17]:
df_C_OT_TIT.sample(5)

,c_ot_tit_chinm,c_ot_tit_engnm,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end,len
c_ot_TIT_id,,,,,,
939,谷王,NaN,NaN,NaN,NaN,2
1053,襄王,NaN,NaN,NaN,NaN,2
1120,德嬪,NaN,NaN,NaN,NaN,2
1024,侍書,NaN,NaN,NaN,NaN,2
1017,寺正,NaN,NaN,NaN,NaN,2
